<a href="https://colab.research.google.com/github/carlalopezz/UFV-visualizacion/blob/main/Ejercicios_clase/Clase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly_express
import plotly.express as px
import pandas as pd

In [2]:
# Lectura de los datos
datos = pd.read_csv('datos_ejercicio_ventas.csv', sep=',')
datos.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


# Preprocesamiento

**1. Número de actuals y de forecasts.**

In [3]:
scenario_counts = datos['SCENARIO'].value_counts()
print(scenario_counts)

SCENARIO
AI_forecast    17766
actual           900
Name: count, dtype: int64


**2. Horizonte de predicción**

Primero tenemos que quitar los actuals ya que no nos interesan. Como el horizonte de predicción es siempre el mismo, podemos poner cualquiera de los valores introducidos, por simplicidad cogemos el primero.

In [4]:
# Mapear los códigos de pronóstico a meses
forecast_map = {
    'AI_P02F': 1, 'AI_P03F': 2, 'AI_P04F': 3, 'AI_P05F': 4,
    'AI_P06F': 5, 'AI_P07F': 6, 'AI_P08F': 7, 'AI_P09F': 8,
    'AI_P10F': 9, 'AI_P11F': 10, 'AI_P12F': 11, 'AI_PF': 12
}

# Agregar una nueva columna para el mes de pronóstico
datos['FORECAST_MONTH'] = datos['FORECAST'].map(forecast_map)

# Función para calcular la diferencia en meses entre la fecha de pronóstico y la fecha de los datos
def calculate_month_diff(row):
    forecast_year = row['FORECAST_YEAR']
    forecast_month = row['FORECAST_MONTH']
    data_year = row['YEAR']
    data_month = row['MONTH']

    # Calcular los meses totales para ambas fechas
    forecast_total_months = forecast_year * 12 + forecast_month
    data_total_months = data_year * 12 + data_month

    # Calcular la diferencia en meses
    return data_total_months - forecast_total_months

# Aplicar la función para calcular la diferencia de meses para cada fila
datos['MONTH_DIFF'] = datos.apply(calculate_month_diff, axis=1)

# Encontrar la diferencia máxima por país
max_diff_per_country = datos.groupby('COUNTRY')['MONTH_DIFF'].max().reset_index()

# Mostrar el resultado
print("Diferencia máxima en meses por país (Horizonte):")
print(max_diff_per_country)

Diferencia máxima en meses por país (Horizonte):
         COUNTRY  MONTH_DIFF
0          Czech        18.0
1        Denmark        18.0
2  Great Britain        18.0
3        Hungary        18.0
4          Italy        18.0
5    Netherlands        18.0
6         Norway        18.0
7       Portugal        18.0
8          Spain        18.0


**3. Nº países y de productos**

In [5]:
num_paises = datos['COUNTRY'].nunique()
num_productos = datos['SUBBRAND'].nunique()

print(f'Número de países diferentes: {num_paises}')
print(f'Número de productos (submarcas) diferentes: {num_productos}')


Número de países diferentes: 9
Número de productos (submarcas) diferentes: 6


**4. Histórico**

El valor más antiguo y el más nuevo.

In [6]:
# ACTUALS
pais_concreto = 'Portugal'
datos_actuals_pais = datos[(datos['COUNTRY'] == pais_concreto) & (datos['SCENARIO'] == 'actual')]

if datos_actuals_pais.empty:
    print(f"No hay datos de ventas para {pais_concreto} en el escenario 'actuals'.")
else:
    anio_minimo = datos_actuals_pais['YEAR'].min()
    anio_maximo = datos_actuals_pais['YEAR'].max()

    meses_anio_minimo = datos_actuals_pais[datos_actuals_pais['YEAR'] == anio_minimo]['MONTH']
    mes_minimo = meses_anio_minimo.min()

    meses_anio_maximo = datos_actuals_pais[datos_actuals_pais['YEAR'] == anio_maximo]['MONTH']
    mes_maximo = meses_anio_maximo.max()

    print(f"Primera venta en {pais_concreto}: {anio_minimo}-{mes_minimo:02d}")
    print(f"Última venta en {pais_concreto}: {anio_maximo}-{mes_maximo:02d}")

Primera venta en Portugal: 2023-01
Última venta en Portugal: 2024-08


**5. Forecasts distintos**

In [7]:
valores_forecast = datos['FORECAST'].unique()

print(f'Los valores únicos de la columna FORECAST son: {valores_forecast}')

Los valores únicos de la columna FORECAST son: ['AI_P02F' 'AI_P10F' 'AI_P09F' 'AI_P03F' 'AI_PF' 'AI_P11F' 'AI_P06F'
 'AI_P05F' 'AI_P07F' 'AI_P12F' 'AI_P08F' 'AI_P04F' nan]


# **1. Cómo se distribuyen las ventas realizadas en:**



In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Filtrar solo los datos reales
data_actuals = datos[datos['SCENARIO'] == 'actual']

# Convertir la columna AMOUNT a formato numérico
data_actuals['AMOUNT'] = data_actuals['AMOUNT'].apply(lambda x: float(str(x).replace(",", ".")))

<ipython-input-8-c1e14d52c1d6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_actuals['AMOUNT'] = data_actuals['AMOUNT'].apply(lambda x: float(str(x).replace(",", ".")))


Según país

In [9]:
# Agrupar y sumar las ventas por país
sales_by_country = data_actuals.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Crear el gráfico de barras
fig = px.bar(
    sales_by_country,
    x='COUNTRY',
    y='AMOUNT',
    title='Distribución de Ventas por País',
    labels={'COUNTRY': 'País', 'AMOUNT': 'Ventas'},
    color='AMOUNT',
    color_continuous_scale='magma'
)

# Personalizar el título y el diseño
fig.update_layout(
    title_font_size=20,
    title_font_family="bold",
    xaxis_title_font_size=14,
    xaxis_title_font_family="bold",
    yaxis_title_font_size=14,
    yaxis_title_font_family="bold",
    template='plotly_white'
)

fig.show()

Según fecha

In [10]:
# Creamos la fecha completa, para trabajar con una sola variable
data_actuals['DATE'] = pd.to_datetime(data_actuals[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar y sumar las ventas por fecha (por mes)
sales_by_date = data_actuals.groupby('DATE')['AMOUNT'].sum().reset_index()

# Crear el gráfico de líneas
fig = px.line(
    sales_by_date,
    x='DATE',
    y='AMOUNT',
    title='Evolución de Ventas Mensuales',
    labels={'DATE': 'Fecha', 'AMOUNT': 'Total de Ventas'},
    markers=True
)

# Personalizar el diseño del gráfico
fig.update_traces(line_color='purple', line_width=2)
fig.update_layout(
    title_font_size=18,
    title_font_family="bold",
    xaxis_title_font_size=14,
    xaxis_title_font_family="bold",
    yaxis_title_font_size=14,
    yaxis_title_font_family="bold",
    template='plotly_white'
)

fig.show()


<ipython-input-10-ef4253d0bf19>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Según cada marca

In [11]:
# De misma forma que con los apartados anteriores
sales_by_brand = data_actuals.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
#print(sales_by_brand)

# Crear el gráfico de barras
fig = px.bar(
    sales_by_brand,
    x='SUBBRAND',
    y='AMOUNT',
    title='Distribución de Ventas por Marca',
    labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Ventas'},
    color_discrete_sequence=px.colors.sequential.Magma
)

# Personalizar el diseño del gráfico
fig.update_layout(
    title_font_size=20,
    title_font_family="bold",
    xaxis_title_font_size=14,
    xaxis_title_font_family="bold",
    yaxis_title_font_size=14,
    yaxis_title_font_family="bold",
    template='plotly_white'
)

fig.show()

# **2. Tendencia y estacionalidad**

Todas las ventas del país con menos ventas

In [12]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Encontrar el país con menos ventas usando el índice del valor mínimo (idxmin)
country_min_sales = sales_by_country.loc[sales_by_country['AMOUNT'].idxmin(), 'COUNTRY']

# Filtrar los datos para el país con menos ventas
data_min_sales_country = data_actuals[data_actuals['COUNTRY'] == country_min_sales]

# Creamos la fecha completa, para trabajar con una sola variable
data_min_sales_country['DATE'] = pd.to_datetime(data_min_sales_country[['YEAR', 'MONTH']].assign(DAY=1))

# Filtrar los datos para el análisis de tendencia y estacionalidad
ventas_pais = data_min_sales_country.groupby(['DATE'])['AMOUNT'].sum()

# Serie temporal
resultados_decomposicion = seasonal_decompose(ventas_pais, model='additive', period=6)

# Crear una figura con 3 filas y una columna
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=(
                        'Ventas Originales (España)',
                        'Tendencia de las Ventas',
                        'Estacionalidad de las Ventas'
                    ),
                    vertical_spacing=0.15)

# Gráfico de la serie original
fig.add_trace(
    go.Scatter(x=ventas_pais.index, y=ventas_pais, mode='lines', name='Ventas Originales', line=dict(color='blue')),
    row=1, col=1
)

# Gráfico de la tendencia
fig.add_trace(
    go.Scatter(x=resultados_decomposicion.trend.index, y=resultados_decomposicion.trend, mode='lines', name='Tendencia', line=dict(color='orange')),
    row=2, col=1
)

# Gráfico de la estacionalidad
fig.add_trace(
    go.Scatter(x=resultados_decomposicion.seasonal.index, y=resultados_decomposicion.seasonal, mode='lines', name='Estacionalidad', line=dict(color='green')),
    row=3, col=1
)

# Configuración de los ejes y diseño
fig.update_xaxes(title_text="Fecha", row=1, col=1)
fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_xaxes(title_text="Fecha", row=3, col=1)

fig.update_yaxes(title_text="Ventas Totales", row=1, col=1)
fig.update_yaxes(title_text="Tendencia", row=2, col=1)
fig.update_yaxes(title_text="Estacionalidad", row=3, col=1)

fig.update_layout(
    height=800,
    title_text="Descomposición de la Serie Temporal de Ventas (España)",
    showlegend=False,
    template="plotly_white"
)

# Mostrar la figura
fig.show()

<ipython-input-12-03cdbaa94309>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



La marca con más ventas

In [13]:
# Encontrar la submarca con más ventas usando el índice del valor máximo
subbrand_max_sales = sales_by_brand.loc[sales_by_brand['AMOUNT'].idxmax(), 'SUBBRAND']

# Filtrar los datos para la submarca con más ventas
data_max_sales_subbrand = data_actuals[data_actuals['SUBBRAND'] == subbrand_max_sales]

# Ordenar los datos de la submarca con más ventas por fecha
data_max_sales_subbrand = data_max_sales_subbrand.sort_values(by='DATE')

# Filtrar los datos para el análisis de tendencia y estacionalidad
ventas_subbrand = data_max_sales_subbrand.groupby(['DATE'])['AMOUNT'].sum()

# Descomposición estacional
resultados_decomposicion = seasonal_decompose(ventas_subbrand, model='additive', period=6)

# Crear una figura con 3 filas para la serie original, tendencia y estacionalidad
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=[
                        f'Ventas Originales ({subbrand_max_sales})',
                        'Tendencia de las Ventas',
                        'Estacionalidad de las Ventas'
                    ],
                    vertical_spacing=0.1)

# Gráfico de la serie original
fig.add_trace(
    go.Scatter(x=ventas_subbrand.index, y=ventas_subbrand, mode='lines', name='Ventas Originales', line=dict(color='blue')),
    row=1, col=1
)

# Gráfico de la tendencia
fig.add_trace(
    go.Scatter(x=resultados_decomposicion.trend.index, y=resultados_decomposicion.trend, mode='lines', name='Tendencia', line=dict(color='orange')),
    row=2, col=1
)

# Gráfico de la estacionalidad
fig.add_trace(
    go.Scatter(x=resultados_decomposicion.seasonal.index, y=resultados_decomposicion.seasonal, mode='lines', name='Estacionalidad', line=dict(color='green')),
    row=3, col=1
)

# Configuración de los ejes y diseño
fig.update_xaxes(title_text="Fecha", row=1, col=1)
fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_xaxes(title_text="Fecha", row=3, col=1)

fig.update_yaxes(title_text="Ventas Totales", row=1, col=1)
fig.update_yaxes(title_text="Tendencia", row=2, col=1)
fig.update_yaxes(title_text="Estacionalidad", row=3, col=1)

fig.update_layout(
    height=800,
    title_text=f"Descomposición de la Serie Temporal de Ventas ({subbrand_max_sales})",
    showlegend=False,
    template="plotly_white"
)

# Mostrar la figura
fig.show()


# **3. Cuáles son las predicciones hechas en España y cómo de buenas son.**

In [16]:
import plotly.graph_objects as go
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Filtrar los datos para España
df_spain = datos[datos['COUNTRY'] == 'Spain'].copy()

# Crear DataFrames separados para las ventas actuales y predichas
df_actual = df_spain[df_spain['SCENARIO'] == 'actual'].copy()
df_forecast = df_spain[df_spain['SCENARIO'] == 'AI_forecast'].copy()

# Crear la columna de fecha
df_actual['DATE'] = pd.to_datetime(df_actual[['YEAR', 'MONTH']].assign(DAY=1))
df_forecast['DATE'] = pd.to_datetime(df_forecast[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupar las ventas reales por mes y año
sales_actual = df_actual.groupby('DATE')['AMOUNT'].sum().reset_index()

# Definir las predicciones bien ajustadas y mal ajustadas
predicciones_ajustadas = ['AI_PF', 'AI_P02F', 'AI_P03F']
predicciones_no_ajustadas = [f for f in df_forecast['FORECAST'].unique() if f not in predicciones_ajustadas]

# Calcular y mostrar MAE para predicciones bien ajustadas
mae_ajustadas = {}
for forecast_type in predicciones_ajustadas:
    forecast_data = df_forecast[df_forecast['FORECAST'] == forecast_type]
    sales_forecast = forecast_data.groupby('DATE')['AMOUNT'].sum().reset_index()
    merged_data = pd.merge(sales_actual, sales_forecast, on='DATE', suffixes=('_actual', '_forecast'))
    mae_ajustadas[forecast_type] = mean_absolute_error(merged_data['AMOUNT_actual'], merged_data['AMOUNT_forecast'])

print("MAE para predicciones bien ajustadas:")
for forecast_type, mae in mae_ajustadas.items():
    print(f"{forecast_type}: {mae}")

# Calcular y mostrar MAE para predicciones no ajustadas
mae_no_ajustadas = {}
for forecast_type in predicciones_no_ajustadas:
    forecast_data = df_forecast[df_forecast['FORECAST'] == forecast_type]
    sales_forecast = forecast_data.groupby('DATE')['AMOUNT'].sum().reset_index()
    merged_data = pd.merge(sales_actual, sales_forecast, on='DATE', suffixes=('_actual', '_forecast'))
    mae_no_ajustadas[forecast_type] = mean_absolute_error(merged_data['AMOUNT_actual'], merged_data['AMOUNT_forecast'])

print("MAE para predicciones no ajustadas:")
for forecast_type, mae in mae_no_ajustadas.items():
    print(f"{forecast_type}: {mae}")

# Primer gráfico: Ventas reales y predicciones bien ajustadas
fig_ajustadas = go.Figure()

# Añadir la línea de ventas reales
fig_ajustadas.add_trace(go.Scatter(
    x=sales_actual['DATE'],
    y=sales_actual['AMOUNT'],
    mode='lines+markers',
    name='Ventas Reales (Actual)',
    line=dict(color='blue')
))

# Añadir cada línea de predicción bien ajustada
for forecast_type in predicciones_ajustadas:
    forecast_data = df_forecast[df_forecast['FORECAST'] == forecast_type]
    sales_forecast = forecast_data.groupby('DATE')['AMOUNT'].sum().reset_index()

    fig_ajustadas.add_trace(go.Scatter(
        x=sales_forecast['DATE'],
        y=sales_forecast['AMOUNT'],
        mode='lines+markers',
        name=f'Predicción {forecast_type} (MAE: {mae_ajustadas[forecast_type]:.2f})',
        line=dict(dash='dash')  # Línea discontinua para diferenciar predicciones de las ventas reales
    ))

# Configuración del primer gráfico
fig_ajustadas.update_layout(
    title='Comparación de Ventas Reales vs. Predicciones Ajustadas (España)',
    xaxis_title='Fecha',
    yaxis_title='Cantidad Ventas',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    template="plotly_white"
)

# Segundo gráfico: Ventas reales y predicciones no ajustadas
fig_no_ajustadas = go.Figure()

# Añadir la línea de ventas reales
fig_no_ajustadas.add_trace(go.Scatter(
    x=sales_actual['DATE'],
    y=sales_actual['AMOUNT'],
    mode='lines+markers',
    name='Ventas Reales (Actual)',
    line=dict(color='blue')
))

# Añadir cada línea de predicción no ajustada
for forecast_type in predicciones_no_ajustadas:
    forecast_data = df_forecast[df_forecast['FORECAST'] == forecast_type]
    sales_forecast = forecast_data.groupby('DATE')['AMOUNT'].sum().reset_index()

    fig_no_ajustadas.add_trace(go.Scatter(
        x=sales_forecast['DATE'],
        y=sales_forecast['AMOUNT'],
        mode='lines+markers',
        name=f'{forecast_type}',
        line=dict(dash='dot')  # Línea punteada para diferenciar predicciones de las ventas reales
    ))

# Configuración del segundo gráfico con leyenda abajo a la derecha
fig_no_ajustadas.update_layout(
    title='Comparación de Ventas Reales vs. Predicciones No Ajustadas (España)',
    xaxis_title='Fecha',
    yaxis_title='Cantidad Ventas',
    legend=dict(
        orientation="h",  # Leyenda en orientación horizontal
        yanchor="bottom",
        y=-0.5,  # Posición vertical (parte inferior)
        xanchor="right",
        x=1  # Posición horizontal (parte derecha)
    ),
    template="plotly_white"
)

# Mostrar los gráficos
fig_ajustadas.show()
fig_no_ajustadas.show()


MAE para predicciones bien ajustadas:
AI_PF: 46005.55791498145
AI_P02F: 58108.79199575566
AI_P03F: 88756.96772535151
MAE para predicciones no ajustadas:
AI_P09F: 771965.2042154823
AI_P05F: 647076.8007885476
AI_P11F: 791807.6399568405
AI_P04F: 637157.4767797586
AI_P12F: 793149.4975949316
AI_P06F: 738663.2382129284
AI_P08F: 740606.2375650405
AI_P07F: 751313.7395571526
AI_P10F: 755036.971687566


Podemos observar que en el primer gráfico, las predicciones realizadas anualmente, en enero y febrero, se ajustan a los valores a los actuals considerablemente bien, con unos valores de MAE bastante bajos en comparación con los tipos de datos que estamos utilizando.
Por otro lado, la segunda gráfica muestra predicciones mal hechas, pues existen valores duplicados y por lo tanto, dan valores más altos. No se representa el MAE sobre el gráfico ya que partimos de la base de que la predicción es errónea.
